# Importing packages

In [ ]:
# Import the necessary packages
import numpy as np
import pandas as pd
import sqlite3

# Declare empty Rotterdam yearly dataframe

In [ ]:
#Create empty dataframe
Rotterdam_2018 = pd.DataFrame()
Rotterdam_2018.shape

# Reading and processing the .db file
Just change the name of the db file in the connection creation and in the export csv action to reuse this logic.
This can be skipped if the CSV files are already created.

In [ ]:
# Connect with the db file and create cursor object to be able to execute queries
connection = sqlite3.connect('2018/month_2018_06.db')
cursor = connection.cursor()

In [ ]:
#Using SQlite for insight into the tables, print one of the existing tables.
#Just for testing purposes, can be skipped.
#for row in cursor.execute('SELECT * FROM tweets_NL'):
        #print(row)

In [ ]:
# Create a pandas dataframe based on a table from the database
tweets_NL = pd.read_sql_query("SELECT * FROM tweets_NL", connection)
users_NL = pd.read_sql_query("SELECT * FROM user_NL", connection)

In [ ]:
# Get an idea of the data
tweets_NL.shape

# Creating a Rotterdam dataframe

In [ ]:
# Drop the columns with missing lat or lon information
tweets_NL = tweets_NL[tweets_NL['geo_lat'].isnull() == False]
tweets_NL = tweets_NL[tweets_NL['geo_lon'].isnull() == False]
tweets_NL.shape

In [ ]:
# Filter all the tweets based on location of the tweet, separating between Rotterdam and Amsterdam

# The lat and lon values of the cities.
lat_Amsterdam = 52.37403
lon_Amsterdam = 4.88969

lat_Rotterdam = 51.9225
lon_Rotterdam = 4.47917

# Create an empty list to fill and eventually append to the existing dataframe
city_inferred = []

for index, row in tweets_NL.iterrows():
    
    # Get the differences in lat and lon values of a certain tweet
    delta_lat_Amsterdam = abs(row['geo_lat'] - lat_Amsterdam)
    delta_lat_Rotterdam = abs(row['geo_lat'] - lat_Rotterdam)
    
    delta_lon_Amsterdam = abs(row['geo_lon'] - lon_Amsterdam)
    delta_lon_Rotterdam = abs(row['geo_lon'] - lon_Rotterdam)
    
    # Based on those differences, assign either Rotterdam or Amsterdam to the tweet
    if delta_lat_Amsterdam > delta_lat_Rotterdam and delta_lon_Amsterdam > delta_lon_Rotterdam:
        city_inferred.append('Rotterdam')
    else: 
        city_inferred.append('Amsterdam')

In [ ]:
# Add the new column to the dataframe
tweets_NL['city_inferred'] = city_inferred

In [ ]:
# Create a Rotterdam dataframe tweets_R
tweets_R = tweets_NL[tweets_NL['city_inferred'] == 'Rotterdam']

#### Combining user information into the table

In [ ]:
tweetsAndUsers_R = pd.merge(tweets_R, users_NL, on='item_number', how='left')

In [ ]:
tweetsAndUsers_R.drop(columns=['profile_image_url', 'profile_background_image_url', 'profile_banner_url', 
                              'url', 'listed_count', 'in_reply_to_user_id', 'in_reply_to_status_id',
                              'quoted_status_id','created_timestamp_y', 'favourites_count', 'statuses_count',
                              'friends_count'], inplace = True)

In [ ]:
# Append the month to the yearly dataframe
Rotterdam_2018 = Rotterdam_2018.append(tweetsAndUsers_R)

In [ ]:
print('shape monthly database:' + str(tweetsAndUsers_R.shape))
print('shape yearly database:' + str(Rotterdam_2018.shape))

# Combining all the monthly databases into one yearly Rotterdam database

In [ ]:
Rotterdam_2018.to_csv('2018.csv', index=False)

In [ ]:
Rotterdam_2018_01 = pd.read_csv('2018.csv', sep= ',', low_memory = False, lineterminator='\n')